Data Cleaning for the Team Project TTC Delay

Objective:

This code prepares the data for analysis by merging datasets for period Jan, 01, 2024 to Jan 31, 2025, cleaning up unnecessary columns, and standardizing date formats. It sets the foundation for further exploration and visualization of TTC subway delays.
The  the result for years 2024 and 2025 datasets are concatenated into a single DataFrame named 'df_delay.csv' and saved at directory  '../data/processed_data/df_delay.csv'.

Files Used:
1. ttc-subway-delay-data-2024.csv:
 - this file contains TTC subway delay data for the year 2024.
 - it is loaded into a DataFrame named df_2024 using pd.read_csv().

2. TTC Subway Delay Data since 2025.csv:
 - this file contains TTC subway delay data for month of January of 2025.

3. code_category_description.csv 
- file to categorize various delay codes related to transportation operations

Data Description
* Date Range: Jan, 01 2024 to Jan 31, 2025
* Company:The Toronto subway, operated by the TTC (Toronto Transit Commission), consists of three lines: Line 1 (Yonge-University), Line 2 (Bloor-Danforth), and Line 4 (Sheppard). 
* Data Points: 28,571 entries 
* Adjustments: 17,653 entries in dataset 'df_delay.csv'
* Additional source: 'code_category_description.csv' (130 entries)


Importing Necessary Libraries:

There are loster essential libraries required for data manipulation, visualization and modeling done in this project:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Machine Learning and Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib

# Deep Learning 
from sklearn.metrics import accuracy_score


Load The Data

In [2]:
# Load the necessary libraries
import numpy as np 
import pandas as pd 
import os

In [3]:
#cload the data for years 2024 and 2025
df_2024 = pd.read_csv("../data/raw_data/ttc-subway-delay-data-2024.csv")

df_2025 = pd.read_csv("../data/raw_data/TTC Subway Delay Data since 2025.csv")

#drop column '_id'
df_2025 = df_2025.drop(columns='_id')

# Converting the 'Date' column in both dataframes to datetime objects.
df_2024['Date'] = pd.to_datetime(df_2024['Date'])
df_2025['Date'] = pd.to_datetime(df_2025['Date'], format='%Y-%m-%d')

# concatename the datast for the 2024, 2025
df = pd.concat([df_2024, df_2025], ignore_index=True)

# Display the first few rows of the dataset
df.head()


,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,2024-01-01,02:00,Monday,SHEPPARD STATION,MUI,0.0,0.0,N,YU,5491.0
1,2024-01-01,02:00,Monday,DUNDAS STATION,MUIS,0.0,0.0,N,YU,0.0
2,2024-01-01,02:08,Monday,DUNDAS STATION,MUPAA,4.0,10.0,N,YU,6051.0
3,2024-01-01,02:13,Monday,KENNEDY BD STATION,PUTDN,10.0,16.0,E,BD,5284.0
4,2024-01-01,02:22,Monday,BLOOR STATION,MUPAA,4.0,10.0,N,YU,5986.0


In [4]:
# Check the data types and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28571 entries, 0 to 28570
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       28571 non-null  datetime64[ns]
 1   Time       28571 non-null  object        
 2   Day        28571 non-null  object        
 3   Station    28571 non-null  object        
 4   Code       28571 non-null  object        
 5   Min Delay  27275 non-null  float64       
 6   Min Gap    27230 non-null  float64       
 7   Bound      18329 non-null  object        
 8   Line       28522 non-null  object        
 9   Vehicle    27754 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 2.2+ MB


In [5]:
# Check unique values
df.nunique()

Date          397
Time         1426
Day             7
Station       473
Code          125
Min Delay     109
Min Gap       106
Bound           5
Line           22
Vehicle       786
dtype: int64

In [6]:
#Removing SRT Line as it is not in use
df = df[df['Line'] != 'SRT']
df['Line'].unique()

array(['YU', 'BD', 'YUS', 'YU/BD', 'SHP', nan, 'BLOOR DANFORTH',
       'YU / BD', 'YU/ BD', 'YUS/BD', 'SHEP', 'LINE 1',
       'TRACK LEVEL ACTIVITY', 'YU & BD', '109 RANEE',
       'ONGE-UNIVERSITY AND BL', 'YU/BD/SHP', 'BD/ YUS', 'BD/ YU',
       'BD/YU', 'BD / YU', '20 CLIFFSIDE'], dtype=object)

In [7]:
 #count isnull values per columns
print(df.isnull().sum())

Date             0
Time             0
Day              0
Station          0
Code             0
Min Delay     1296
Min Gap       1341
Bound        10240
Line            49
Vehicle        817
dtype: int64


The "Vehicle" column, which provides the registered number of the train conducting the TTC trip, is deemed irrelevant for the purposes of our analysis. Consequently, this data may be considered for removal.

In [9]:
df =  df.drop('Vehicle',axis=1)

Handling Missing Values

Given the dataset containing 28,572 entries, we have opted to address missing values by removing rows with null values. This decision is informed by the observation that the column "Bound" possesses the largest number of missing entries, totaling 10,240. While this represents a significant portion of the missing data, it constitutes only a fraction of the overall dataset. Consequently, the removal of these entries is expected to have minimal impact on the integrity and representativeness of the analysis

In [11]:
 #count isnull values per columns
print(df.isnull().sum())

Date             0
Time             0
Day              0
Station          0
Code             0
Min Delay     1296
Min Gap       1341
Bound        10240
Line            49
dtype: int64


In [12]:
#drop isnull values in Min Delay and columns
df_cleaned = df.dropna() #Nan  0


In [14]:
df_cleaned.info()   #28571    17695 

<class 'pandas.core.frame.DataFrame'>
Index: 17695 entries, 0 to 28570
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       17695 non-null  datetime64[ns]
 1   Time       17695 non-null  object        
 2   Day        17695 non-null  object        
 3   Station    17695 non-null  object        
 4   Code       17695 non-null  object        
 5   Min Delay  17695 non-null  float64       
 6   Min Gap    17695 non-null  float64       
 7   Bound      17695 non-null  object        
 8   Line       17695 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 1.4+ MB


In [15]:
 #count isnull values per column
print(df_cleaned.isnull().sum())

Date         0
Time         0
Day          0
Station      0
Code         0
Min Delay    0
Min Gap      0
Bound        0
Line         0
dtype: int64


Additional features

To enhance the dataset, additional features were derived from the existing "Date" column. 
Specifically:
- a new column labeled "Month" was created, extracting the numerical representation of the month (ranging from 1 to 12) from the "Date" values.
- another column titled "Month Name" was added, containing the full textual names of the months (e.g., January, February) corresponding to the "Date" values.

In [17]:
# Creating new features from 'Date' column
df_cleaned['Month'] = df_cleaned['Date'].dt.month

# Adding a new column with month names
df_cleaned['Month Name'] = df_cleaned['Date'].dt.month_name()

print(df_cleaned.head())

        Date   Time     Day             Station   Code  Min Delay  Min Gap  \
0 2024-01-01  02:00  Monday    SHEPPARD STATION    MUI        0.0      0.0   
1 2024-01-01  02:00  Monday      DUNDAS STATION   MUIS        0.0      0.0   
2 2024-01-01  02:08  Monday      DUNDAS STATION  MUPAA        4.0     10.0   
3 2024-01-01  02:13  Monday  KENNEDY BD STATION  PUTDN       10.0     16.0   
4 2024-01-01  02:22  Monday       BLOOR STATION  MUPAA        4.0     10.0   

  Bound Line  Month Month Name  
0     N   YU      1    January  
1     N   YU      1    January  
2     N   YU      1    January  
3     E   BD      1    January  
4     N   YU      1    January  


C:\Users\User\AppData\Local\Temp\ipykernel_2712\1454598829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Month'] = df_cleaned['Date'].dt.month
C:\Users\User\AppData\Local\Temp\ipykernel_2712\1454598829.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Month Name'] = df_cleaned['Date'].dt.month_name()


The "Code" column, which identifies the specific reasons for TTC delays, has been categorized to streamline the analytical process. The categorization was informed by information sourced from online references, which was subsequently incorporated into the dataset titled 'code_category_description.csv'. This approach aims to enhance the clarity and efficiency of the analysis by grouping similar delay codes under broader, defined categories.

In [19]:
# add dataset for Code categories
df_code = pd.read_csv("../data/raw_data/code_category_description.csv")
print(df_code)


    Delay Code                                 Category  \
0         EUAC  Mechanical/Electrical/Vehicle Equipment   
1         EUAL  Mechanical/Electrical/Vehicle Equipment   
2        EUATC  Mechanical/Electrical/Vehicle Equipment   
3         EUBK  Mechanical/Electrical/Vehicle Equipment   
4         EUBO  Mechanical/Electrical/Vehicle Equipment   
..         ...                                      ...   
124        TUS                  Transportation/Operator   
125       TUSC                  Transportation/Operator   
126      TUSET                  Transportation/Operator   
127       TUST                         Weather/External   
128      TUSUP                  Transportation/Operator   

                                           Description  
0           Train HVAC malfunction or underperformance  
1      Issues with onboard or wayside AC power systems  
2    ATC equipment malfunction related to Rail Cars...  
3          Faulty brake components or pneumatic issues  
4     

In [20]:
# Check unique values
print("\nUnique Values per Column:")
df_code.nunique()


Unique Values per Column:


Delay Code     129
Category        11
Description    129
dtype: int64

In [21]:
df_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Delay Code   129 non-null    object
 1   Category     129 non-null    object
 2   Description  129 non-null    object
dtypes: object(3)
memory usage: 3.1+ KB


In [22]:
# Adding 'Category' column from df_code to df_cleanese based on 'Delay code'
df_delay = pd.merge(df_cleaned, df_code, left_on='Code', right_on = 'Delay Code',how='inner')

In [ ]:
df_delay.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17653 entries, 0 to 17652
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         17653 non-null  datetime64[ns]
 1   Time         17653 non-null  object        
 2   Day          17653 non-null  object        
 3   Station      17653 non-null  object        
 4   Code         17653 non-null  object        
 5   Min Delay    17653 non-null  float64       
 6   Min Gap      17653 non-null  float64       
 7   Bound        17653 non-null  object        
 8   Line         17653 non-null  object        
 9   Month        17653 non-null  int32         
 10  Month Name   17653 non-null  object        
 11  Delay Code   17653 non-null  object        
 12  Category     17653 non-null  object        
 13  Description  17653 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(1), object(10)
memory usage: 1.8+ MB


In [25]:
 #count isnull values per column
print(df_delay.isnull().sum())

Date           0
Time           0
Day            0
Station        0
Code           0
Min Delay      0
Min Gap        0
Bound          0
Line           0
Month          0
Month Name     0
Delay Code     0
Category       0
Description    0
dtype: int64


The columns "Delay Code" and "Description" contain redundant information and may be considered for removal from the dataset to enhance clarity and reduce unnecessary duplication.

In [ ]:
#drop redundant columns
df_delay =  df_delay.drop(['Delay Code','Description'],axis=1)

In [375]:
#sort by date, time
df_delay = df_delay.sort_values(by=['Date', 'Time'])

In [376]:
# Printing new dataframe after creation of new features
print(df_delay.head())

         Date   Time     Day                Station   Code  Min Delay  \
38 2024-01-01  00:44  Monday     VAUGHAN MC STATION  MUPAA        0.0   
39 2024-01-01  00:48  Monday          BLOOR STATION    SUO        0.0   
40 2024-01-01  01:10  Monday     VAUGHAN MC STATION    MUO        8.0   
41 2024-01-01  01:11  Monday         DUNDAS STATION    SUO        0.0   
42 2024-01-01  01:38  Monday  ST GEORGE YUS STATION   SUUT        0.0   

    Min Gap Bound Line  Month Month Name                           Category  
38      0.0     S   YU      1    January  Door/Passenger/Platform Incidents  
39      0.0     S   YU      1    January                  Security/Policing  
40     14.0     S   YU      1    January                      Miscellaneous  
41      0.0     S   YU      1    January                  Security/Policing  
42      0.0     N   YU      1    January                  Security/Policing  


In [377]:
# Specify the file path where the dataset should be saved
file_path = '../data/processed_data/df_delay.csv'  

# Save the dataset as a CSV file
df_delay.to_csv(file_path, index=False)

print(f"Dataset has been saved to: {file_path}")

Dataset has been saved to: ../data/processed_data/df_delay.csv
